In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Simulo el dia actual para la recomendacion, porque la red podria inferir diferente dado que como entrada esta el mes.
# La fecha esta basada en la ultima pelicula que se vio de la bd
recommendation_timestamp = 893286638

cantidad_recomendaciones = 10

In [3]:
import pandas as pd
import numpy as np
import sys
from datetime import datetime

In [4]:
PATH_PROCESSED='processed/'
PATH = 'dataset/'

user, occupation = [pd.read_csv(PATH + fname+'.csv', low_memory=False, sep="|") for fname in ['user', 'occupation']]
info, train = [pd.read_csv(PATH + fname+'.csv', low_memory=False, sep="\t") for fname in ['info', 'train']]
test = pd.read_csv(PATH + 'test.csv', low_memory=False, sep=",")
item = pd.read_csv(PATH + 'item.csv',  low_memory=False, encoding='latin-1', sep="|")

item.rename( columns={'movieId' :'movie_id', 'movieTitle' :'movie_title', 'release date' :'release_date', 'video release date' :'video_release_date', 'IMDb URL' :'imdb_url', 'unknown' :'unknown', 'Action' :'action', 'Adventure' :'adventure', 'Animation' :'animation', 'Children\'s' :'children', 'Comedy' :'comedy', 'Crime' :'crime', 'Documentary' :'documentary', 'Drama' :'drama', 'Fantasy' :'fantasy', 'Film-Noir' :'film-noir', 'Horror' :'horror', 'Musical' :'musical', 'Mystery' :'mystery', 'Romance' :'romance', 'Sci-Fi' :'sci-fi', 'Thriller' :'thriller', 'War' :'war', 'Western' :'western'}, inplace=True)

user.rename( columns={'user id' :'user_id', 'age' :'age', 'gender' :'gender', 'occupation' :'occupation', 'zip code' :'zip_code'}, inplace=True)

train.rename( columns={'userId' :'user_id', 'movieId' :'movie_id'}, inplace=True)
test.rename( columns={'Id' :'id', 'userId' :'user_id', 'movieId' :'movie_id'}, inplace=True)


data = pd.read_feather(PATH_PROCESSED + 'joined.fth')


u_unique = data.user_id.unique()
user2Idx = {o:i+1 for i,o in enumerate(u_unique)}

m_unique = data.movie_id.unique()
movie2Idx = {o:i+1 for i,o in enumerate(m_unique)}

In [5]:
genre = ['action', 'adventure', 'animation',
         "children", 'comedy', 'crime', 'documentary', 'drama', 'fantasy',
         'film-noir', 'horror', 'musical', 'mystery', 'romance', 'sci-fi',
         'thriller', 'war', 'western']

In [6]:
data.head()

,user_id,movie_id,rating,timestamp,movie_title,release_date,unknown,action,adventure,animation,...,gender,occupation,zip_code,ReleaseYear,ReleaseMonthYear,WatchedYear,WatchedMonthYear,watchReleaseDiff,watchNow-releaseDiff,ageRange
0,1,1,5,-1.616731,Toy Story (1995),1995-01-01,0,0,0,1,...,1,19,85711,-135.559629,82,0,7,-0.501916,-0.482847,1
1,1,2,3,-1.254339,GoldenEye (1995),1995-01-01,0,1,1,0,...,1,19,85711,-135.559629,82,0,6,-0.497500,-0.482847,1
2,1,3,4,-0.944145,Four Rooms (1995),1995-01-01,0,0,0,0,...,1,19,85711,-135.559629,82,0,5,-0.493852,-0.482847,1
3,1,4,3,-1.254348,Get Shorty (1995),1995-01-01,0,1,0,0,...,1,19,85711,-135.559629,82,0,6,-0.497500,-0.482847,1
4,1,5,3,1.163327,Copycat (1995),1995-01-01,0,0,0,0,...,1,19,85711,-135.559629,82,1,4,-0.469086,-0.482847,1


In [7]:
data[data['user_id'] == 599].iloc[0]

#data[data['movie_id'] == 599]

item[item['movie_id'] == 599]

,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,children,...,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
598,599,Police Story 4: Project S (Chao ji ji hua) (1993),16-Aug-1996,NaN,http://us.imdb.com/M/title-exact?Project%20S%2...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Preparo la entrada de la red con TEST

In [8]:
user_idxs, movie_idxs, timestamps, genres, user_genders, user_occupations, watchedReleaseDiffs, ageRanges, releaseYears = [],[],[],[],[],[],[],[],[]
genres = data[data['movie_id'] == -1].head(1)[genre]

In [9]:
input_size = len(test)

In [10]:
%%time
# Crea las inputs de la nueva red
time_now = data.timestamp.max()
movies_no_rates = []
for index, row in test.iterrows():
    movieData = None
    userData = None
    try:
        userData = data[data['user_id'] == row.user_id].iloc[0]
    except:
        print("Cant find user " + str(row.user_id))
        continue
    try:
        movieData = data[data['movie_id'] == row.movie_id].iloc[0]
    except:
        print("Cant find " + str(index) + " movie: " + str(row.movie_id))
        movies_no_rates += [{'idx': index, 'movie_id': row.movie_id}]
        continue
    user_idxs += [row.user_id]
    movie_idxs += [row.movie_id]
    timestamps += [time_now]
    genres = genres.append(movieData[genre])
    user_genders += [userData.gender]
    user_occupations += [userData.occupation]
    watchedReleaseDiffs += [userData.watchReleaseDiff]
    ageRanges += [userData.ageRange]
    releaseYears += [userData.ReleaseYear]

Cant find 531 movie: 599
Cant find 697 movie: 711
Cant find 1016 movie: 814
Cant find 1022 movie: 830
Cant find 1031 movie: 852
Cant find 1034 movie: 857
Cant find 3639 movie: 1156
Cant find 5063 movie: 1236
Cant find 7414 movie: 1309
Cant find 7415 movie: 1310
Cant find 8025 movie: 1320
Cant find 8035 movie: 1343
Cant find 8039 movie: 1348
Cant find 8046 movie: 1364
Cant find 8051 movie: 1373
Cant find 10632 movie: 1457
Cant find 10633 movie: 1458
Cant find 13031 movie: 1492
Cant find 13032 movie: 1493
Cant find 13035 movie: 1498
Cant find 13633 movie: 1505
Cant find 15577 movie: 1520
Cant find 18319 movie: 1533
Cant find 18443 movie: 1536
Cant find 19004 movie: 1543
Cant find 19222 movie: 1557
Cant find 19223 movie: 1561
Cant find 19224 movie: 1562
Cant find 19225 movie: 1563
Cant find 19226 movie: 1565
Cant find 19228 movie: 1582
Cant find 19229 movie: 1586
CPU times: user 1min 58s, sys: 9.6 s, total: 2min 7s
Wall time: 2min


In [11]:
max_rating = 5
min_rating = 1

# Verify

In [12]:
inputs_val = [user_idxs, movie_idxs, timestamps, pd.DataFrame(genres),
              user_genders, user_occupations, watchedReleaseDiffs, ageRanges, releaseYears]

In [13]:
# Convert input data into series
for i in range(len(inputs_val)):
    if (i != 3):
        inputs_val[i] = pd.Series(inputs_val[i])

In [14]:
# Verify input values
for i in range(len(inputs_val)):
    if (i == 3):
        print(i, type(inputs_val[i]))
    else:
        print(i, type(inputs_val[i]), len(inputs_val[i]), type(inputs_val[i][0]), inputs_val[i][0])

0 <class 'pandas.core.series.Series'> 19968 <class 'numpy.int64'> 1
1 <class 'pandas.core.series.Series'> 19968 <class 'numpy.int64'> 6
2 <class 'pandas.core.series.Series'> 19968 <class 'numpy.float64'> 1.82796423103278
3 <class 'pandas.core.frame.DataFrame'>
4 <class 'pandas.core.series.Series'> 19968 <class 'numpy.int64'> 1
5 <class 'pandas.core.series.Series'> 19968 <class 'numpy.int64'> 19
6 <class 'pandas.core.series.Series'> 19968 <class 'numpy.float64'> -0.5019160586050823
7 <class 'pandas.core.series.Series'> 19968 <class 'numpy.int64'> 1
8 <class 'pandas.core.series.Series'> 19968 <class 'numpy.float64'> -135.55962914359867


0 <class 'pandas.core.series.Series'> 1323 <class 'numpy.int32'> 310
1 <class 'pandas.core.series.Series'> 1323 <class 'numpy.int64'> 1
2 <class 'pandas.core.series.Series'> 1323 <class 'numpy.float64'> 1.825991373017527
3 <class 'pandas.core.frame.DataFrame'>
4 <class 'pandas.core.series.Series'> 1323 <class 'numpy.int32'> 1
5 <class 'pandas.core.series.Series'> 1323 <class 'numpy.int32'> 18
6 <class 'pandas.core.series.Series'> 1323 <class 'numpy.float64'> -0.6338023931125223
7 <class 'pandas.core.series.Series'> 1323 <class 'numpy.int32'> 1
8 <class 'pandas.core.series.Series'> 1323 <class 'numpy.float64'> -135.52115597491908

# Levantar la red

In [15]:
import tensorflow as tf
from keras import Model
import keras.backend as K
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [16]:
model = tf.keras.models.load_model(PATH_PROCESSED + 'fv_model.h5', custom_objects={"root_mean_squared_error": root_mean_squared_error })
#model.load_weights(PATH_PROCESSED + 'fv_weights.hdf5')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Item (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 User (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 gender (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 occupation (InputLayer)        [(None, 1)]          0           []                               
                                                                                            

In [17]:
model.compile('adam', loss= 'mean_squared_error', metrics=[root_mean_squared_error])

# Prediccion Test de la red

In [18]:
predictions = model.predict(inputs_val)

624/624 [==============================] - 1s 1ms/step


In [19]:
predictions

array([[3.5273888],
       [3.5273888],
       [3.5273888],
       ...,
       [3.5273888],
       [3.5273888],
       [3.5273888]], dtype=float32)

In [20]:
print(predictions.max(), predictions.min())
#predictions.head()

3.5273888 3.5273888


# Elegir movies parecidas a las sin rating

In [21]:
from sklearn.neighbors import NearestNeighbors
suggestions_count=1

In [22]:
len(movies_no_rates)

32

In [23]:
movie_embeddings_layer = model.layers[5]
movie_embeddings_layer.name

'Movie-Embedding'

In [24]:
movie_embeddings_matrix = movie_embeddings_layer.get_weights()[0]
movie_embeddings_matrix.shape

(1683, 20)

In [25]:
nbrs_movies = NearestNeighbors(n_neighbors=suggestions_count, metric='cosine').fit(movie_embeddings_matrix)

In [26]:
# Tomo las peliculas similares a las que vio con buen promedio
recommended_movies_similar_watched = []

for row in movies_no_rates:
    for mov in nbrs_movies.kneighbors([movie_embeddings_matrix[row['movie_id']]])[1][0]:
        recommended_movies_similar_watched += [{'idx': row['idx'], 'movie_id': row['movie_id'], 'rec': mov}]
recommended_movies_similar_watched

[{'idx': 531, 'movie_id': 599, 'rec': 599},
 {'idx': 697, 'movie_id': 711, 'rec': 711},
 {'idx': 1016, 'movie_id': 814, 'rec': 814},
 {'idx': 1022, 'movie_id': 830, 'rec': 830},
 {'idx': 1031, 'movie_id': 852, 'rec': 852},
 {'idx': 1034, 'movie_id': 857, 'rec': 857},
 {'idx': 3639, 'movie_id': 1156, 'rec': 1156},
 {'idx': 5063, 'movie_id': 1236, 'rec': 1236},
 {'idx': 7414, 'movie_id': 1309, 'rec': 1309},
 {'idx': 7415, 'movie_id': 1310, 'rec': 1310},
 {'idx': 8025, 'movie_id': 1320, 'rec': 0},
 {'idx': 8035, 'movie_id': 1343, 'rec': 1343},
 {'idx': 8039, 'movie_id': 1348, 'rec': 1348},
 {'idx': 8046, 'movie_id': 1364, 'rec': 1364},
 {'idx': 8051, 'movie_id': 1373, 'rec': 1373},
 {'idx': 10632, 'movie_id': 1457, 'rec': 1457},
 {'idx': 10633, 'movie_id': 1458, 'rec': 1458},
 {'idx': 13031, 'movie_id': 1492, 'rec': 1492},
 {'idx': 13032, 'movie_id': 1493, 'rec': 1493},
 {'idx': 13035, 'movie_id': 1498, 'rec': 1498},
 {'idx': 13633, 'movie_id': 1505, 'rec': 1505},
 {'idx': 15577, 'movie_i

In [27]:
# Genera unica lista con el orden original
default_rating = 2
final_predictions = []
index_similar = 0
index_red = 0
for index, row in test.iterrows():
    new_row = None
    if index_similar < len(recommended_movies_similar_watched):
        similar_row = recommended_movies_similar_watched[index_similar]
        if row.movie_id == similar_row['movie_id']:
            new_row = {'Id': row.id, 'rating': default_rating}
            index_similar +=1
        else:
            new_row = {'Id': row.id, 'rating': predictions[index_red][0]}
            index_red += 1
    else:
        new_row = {'Id': row.id, 'rating': predictions[index_red][0]}
        index_red += 1
    final_predictions += [new_row]

In [28]:
final_predictions = pd.DataFrame(final_predictions, columns=['Id', 'rating'])
print(final_predictions.shape)
final_predictions.head()

(20000, 2)


,Id,rating
0,0,3.527389
1,1,3.527389
2,2,3.527389
3,3,3.527389
4,4,3.527389


In [29]:
# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

# Merge y preparo DF para submit

# Sumbit a la competición

In [30]:
# Your submission should be a CSV file with 20000 rows and a header. You can upload a zip/gz/7z archive.

final_predictions.to_csv(PATH_PROCESSED + f'submision.csv', index=False)

In [31]:
# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

date and time = 31/12/2022 09:55:46
